In [1]:
import pandas as pd

# Data Cleaning

In [2]:
df = pd.read_csv("TMDB Movie Dataset.csv")
df.head()

,title,vote_average,vote_count,release_date,revenue,runtime,budget,popularity,genres
0,Inception,8.364,34495,2010-07-15,825532764,148,160000000,83.952,"Action, Science Fiction, Adventure"
1,Interstellar,8.417,32571,2014-11-05,701729206,169,165000000,140.241,"Adventure, Drama, Science Fiction"
2,The Dark Knight,8.512,30619,2008-07-16,1004558444,152,185000000,130.643,"Drama, Action, Crime, Thriller"
3,Avatar,7.573,29815,2009-12-15,2923706026,162,237000000,79.932,"Action, Adventure, Fantasy, Science Fiction"
4,The Avengers,7.710,29166,2012-04-25,1518815515,143,220000000,98.082,"Science Fiction, Action, Adventure"


The first step is to keep only the first genre listed under the 'genres' column. After examining the dataset, it is clear that they are not listed in alphabetical order so I will move forward with the assumption that the genre listed first is the one most applicable to the movie.

In [3]:
# Edit 'genres' so only the first genre in the list is kept
df['genres'] = df['genres'].astype(str).str.split(',').str[0].str.strip()
df.head()

,title,vote_average,vote_count,release_date,revenue,runtime,budget,popularity,genres
0,Inception,8.364,34495,2010-07-15,825532764,148,160000000,83.952,Action
1,Interstellar,8.417,32571,2014-11-05,701729206,169,165000000,140.241,Adventure
2,The Dark Knight,8.512,30619,2008-07-16,1004558444,152,185000000,130.643,Drama
3,Avatar,7.573,29815,2009-12-15,2923706026,162,237000000,79.932,Action
4,The Avengers,7.710,29166,2012-04-25,1518815515,143,220000000,98.082,Science Fiction


The second step is to remove entries which do not give us the full information. For example, if there is no genre listed, no title listed, or there is an entry of '0' for any of the quantitative details, then that entire row is to be removed.

In [4]:
# Drop rows with missing or incomplete data

# Drop rows that don't give any information for the qualitative data
df_cleaned = df.dropna(subset=['genres', 'title'])

# Drop rows which have don't give sufficient quantitative data
for col in ['vote_count','vote_average','revenue','runtime','budget','popularity']:
    df_cleaned = df_cleaned[df_cleaned[col] > 0]

df_cleaned.head()

,title,vote_average,vote_count,release_date,revenue,runtime,budget,popularity,genres
0,Inception,8.364,34495,2010-07-15,825532764,148,160000000,83.952,Action
1,Interstellar,8.417,32571,2014-11-05,701729206,169,165000000,140.241,Adventure
2,The Dark Knight,8.512,30619,2008-07-16,1004558444,152,185000000,130.643,Drama
3,Avatar,7.573,29815,2009-12-15,2923706026,162,237000000,79.932,Action
4,The Avengers,7.710,29166,2012-04-25,1518815515,143,220000000,98.082,Science Fiction


We now will edit 'release_date' so that only the year is listed since it is the only meaningful aspect.

In [5]:
# Edit 'release_date' so that only the release year is visible

# Convert to datetime format
df_cleaned['release_date'] = pd.to_datetime(df_cleaned['release_date'], errors='coerce')

# Create new column for all the extracted years
df_cleaned['release_year'] = df_cleaned['release_date'].dt.year

# Drop rows which do not have a release year listed
df_cleaned = df_cleaned.dropna(subset=['release_year'])

# Convert to integer
df_cleaned['release_year'] = df_cleaned['release_year'].astype(int)

# Drop release_date column
df_cleaned = df_cleaned.drop(columns=['release_date'])

df_cleaned.head()

,title,vote_average,vote_count,revenue,runtime,budget,popularity,genres,release_year
0,Inception,8.364,34495,825532764,148,160000000,83.952,Action,2010
1,Interstellar,8.417,32571,701729206,169,165000000,140.241,Adventure,2014
2,The Dark Knight,8.512,30619,1004558444,152,185000000,130.643,Drama,2008
3,Avatar,7.573,29815,2923706026,162,237000000,79.932,Action,2009
4,The Avengers,7.710,29166,1518815515,143,220000000,98.082,Science Fiction,2012


In [6]:
print(len(df))
print(len(df_cleaned))
print((len(df_cleaned)/len(df))*100)

1048575
10353
0.9873399613761534


From the above, we can see that our data cleaning only preserved 1% of the rows. However, considering the dataset itself had over 1 million entries where most had missing information, I do not think it is a problem. In fact, we are sure that the data that is still available is fully complete and will allow for meaningful analysis

In [7]:
# Save the cleaned dataset
df_cleaned.to_csv("Cleaned TMDB Dataset.csv", index = False)